<h1><center><b>Project 1: Sequence Labeling</b></center></h1>

---



---



# Introduction
In this project we explore sequence labeling problems in NLP: (i) *Part of Speech (POS) tagging;* and (ii) *Named Entity Recognition* (NER). We first use the Hidden Markov Model (HMM) to resolve the problem of POS tagging. We then apply Conditional Random Fields (CRF) to find solutions for NER.

We uncover some important aspects for NER that require external (human) intervention, for example, the choice of features.

* A good read for this project could be **Chapter 17: Sequence Labeling for Parts of Speech and Named Entities of Jurafsky & Martin's Speech and Language Processing book (SLP3)**.

### Workflow

* Part 0 sets up the environment through installation of the required dependencies and incorporates a kernel restart once the installation finishes.
* Part 1 downloads and explores the dataset.
* Part 2 investigates the application of Viterbi to Part of Speech tagging.
* Part 3 develops a CRF model for NER tasks.
* Part 4 provides instructions for uploading responses and information about grading.

Each of the major parts of this project includes both programming questions (in Gradescope) and quiz questions (in Canvas) that must be answered in order to receive full credit for this assignment.


**Important note on grading**:
1. **Project 1 Programming questions** (in Gradescope) - Some of the functions are incomplete; complete them and pass the associated test cases. Passing a test case does not guarantee full credit. Update your answers in the .py file provided to you and upload it to Gradescope for grading.

2. **Project 1 Quiz questions** (in Canvas) - This project is accompanied by a project quiz. For each project quiz question, select one or more correct options and explain the response in 200 characters or less. Any quiz response that does not include an explanation will not be graded. Explanations must be no longer than 200 characters; any additional words will result in automatic truncation and will not be considered for grading.

**Review Materials for Project 1: Weeks 3-7**
* Videos/handouts
* FTF slides
* Readings per the syllabus: SLP 3, 17, 18, 19, 13.5 ("chunking" from SLP edition 2), Universal Dependencies, LDC Penn Treebank, NLTK 0, 2, 7, Scikit Learn, CRFSuite.


# Part 0: Setting up the environment

## 0.1 Running cells

You can run the code in a cell by pressing (CMD/CTRL + ENTER) or by clicking the run button in the top left corner.

Cells are divided into two categories:
1. Markdown - This is a Markdown cell, which is used to provide instructions, similar to a running readme of your code.
2. Code cell - The Code cell contains all the code.

**Important Points:**

**Jupyter Notebook cells are executed in order - from top to bottom. Any other order is not guaranteed to produce expected results.**

**You must run all the cells in this notebook. (Even if you are not editing them)**

## 0.2 Install Dependencies.

The dependencies listed below help us in developing NLP solutions without having to write a lot of code from scratch.

The compatible version of scikit-learn package with sklearn-crfsuite is less than 24.0, as higher versions may have Attribute_Errors.

Please feel free to learn more about the scikit-learn: https://scikit-learn.org/stable/ and sklearn_crfsuite: https://sklearn-crfsuite.readthedocs.io/en/latest/

**Run the code cell below to install required dependencies.**

> After installation, you may need to restart the kernel. This can be done by selecting `Restart runtime` in the `Runtime` drop-down menu. But here for your convenience, the following code cell is automatically prompted to restart the kernel after installation is finished.

### If you use **google colab**, keep these lines below (lines 3-4) uncommented
### If you use **hipergator**, comment out these lines below (lines 3-4)


In [109]:
#@title Run this cell to install required dependencies.

!pip install -U scikit-learn
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/MeMartijn/updated-sklearn-crfsuite.git to /scratch/local/46902106/pip-install-2dwae1fq/sklearn-crfsuite_1ad65d5ef99a4ad893d3d6167e40d6e6
  Running command git clone --filter=blob:none --quiet https://github.com/MeMartijn/updated-sklearn-crfsuite.git /scratch/local/46902106/pip-install-2dwae1fq/sklearn-crfsuite_1ad65d5ef99a4ad893d3d6167e40d6e6
  Resolved https://github.com/MeMartijn/updated-sklearn-crfsuite.git to commit 675038761b4405f04691a83339d04903790e2b95
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.1.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


# Part 1: Getting Familiar with the Dataset

**NLTK:** There are many datasets in **Natural Language Toolkit (NLTK)** library of Python (see https://www.nltk.org). The two most widely used existing NLTK corpora are:

1. Penn TreeBank Corpus
2. CoNLL Named Entity (NE) Chunk Corpus

We import the NLTK library to access these built-in datasets. To learn more about these and other corpora, see **NLTK Chapter 2: Accessing Text Corpora and Lexical Resources** https://www.nltk.org/book/ch02.html and **NLTK Chapter 7: Extracting Information from Text** https://www.nltk.org/book/ch07.html.


## 1.1 Penn Treebank Corpus
> The completion of this section prepares you to answer question 1 and 2 on **Project 1 Quiz questions**.

The English Penn Treebank (PTB) corpus, and in particular the section of the corpus corresponding to the articles of *Wall Street Journal (WSJ)*, is one of the most well-known and used corpus for the evaluation of models for sequence labeling. In this project, we use this corpus for annotating each word with its *Part-of-Speech tag*.
* A good read for this annotated corpus is **Marcinkiewicz, M. A. (1994). Building a large annotated corpus of English: The Penn Treebank. Using Large Corpora, 273.** https://dl.acm.org/doi/pdf/10.5555/972470.972475

The publicly available **Penn Treebank** in NLTK library is a subset of the corpus (you can buy the entire Treebank, if you want, but you'll have to invest some $700~). This corpus was released by the University of Pennsylvania and contains 36 POS tags and 12 other tags (for punctuation and currency symbols).
* Refer to the complete **list of POS tags for Penn-Treebank** here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

As presented in the lecture, the formalism that underlies the POS tags assigned in the Penn Treebank is the *constituency tree*, which captures *syntactic categories* (i.e., parts of speech) and relationships among words in a sentence.

* This information is covered in **SLP3 17, 18**.

However, in this project, we focus specifically on POS annotation at the word level, rather than sentence-level syntactic structure.

**Run the code below to download and examine the Penn Treebank corpus**


In [110]:
#@title Run this cell to download and examine the Penn Treebank corpus. (Do not edit the code.)
#This cell loads the Penn Treebank corpus from nltk and explores its structure.

#No need to install nltk in google colab since it is preloaded in the environments.
#!pip install nltk
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

#Ensure that the treebank corpus is downloaded
nltk.download('treebank')

#Load the treebank corpus class
from nltk.corpus import treebank

#Now we iterate over all samples from the corpus (the fileids - that are equivalent to sentences)
#and retrieve the word and the pre-labeled PoS tag.

for fileid in treebank.fileids()[:5]:
  print(treebank.tagged_words(fileid))

print(len(treebank.fileids()))


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ...]
[('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ...]
[('A', 'DT'), ('form', 'NN'), ('of', 'IN'), ...]
[('Yields', 'NNS'), ('on', 'IN'), ...]
[('J.P.', 'NNP'), ('Bolduc', 'NNP'), (',', ','), ...]
199


[nltk_data] Downloading package treebank to
[nltk_data]     /home/carson.johnson/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


In Part 2, we will see how the Hidden Markov Model is used for POS tagging over Penn Treebank Corpus.

Hidden Markov Model (HMM) is a probabilistic sequence model: given a sequence of units, it computes a probability distribution over possible sequences of labels and chooses the best label sequence.
* A good read on HMM is **Section 17.4: HMM Part-of-Speech Tagging of SLP3**.



### 1.1.1. An Alternative Annotation Scheme: Universal Dependencies

An alternative format presented in the lectures is the *dependency tree* representation, which is intended to provide a logical form that captures semantic relationships. This directly contrasts with the constituency tree formalism assumed in the Penn Treebank.

At the present time (as of Jan 2022), there are just over 200 treebanks of more than 100 languages available in the Universal dependencies inventory, which has *17 tags in its tagset*: https://universaldependencies.org/u/pos/, in contrast to the larger PennTreebank tag set found here: https://universaldependencies.org/tagset-conversion/en-penn-uposf.html.  The aim of such resources is to achieve cross-linguistic consistency of annotation, while still permitting language-specific extensions when necessary.

Although this project uses PennTreebank Corpus, understanding the difference between the two formalisms is an essential objective of this NLP class.

## 1.2. CoNLL Corpus - 2002

The CONLL-2002 dataset covers two languages: Spanish and Dutch. The Spanish dataset is a collection of newswire articles made available by the Spanish EFE News Agency. The articles are from May 2000. The tagged dataset contains words and entity tags only. The Dutch data consist of four editions of the Belgian newspaper "De Morgen" of 2000 (June 2, July 1, August 1 and September 1).

* A good read is **Sang, E. F. T. K., & De Meulder, F. (1837). Introduction to the CoNLL-2003 Shared Task: Language-Independent Named Entity Recognition. Development, 922, 1341.** https://aclanthology.org/W02-2024.pdf. about the CoNLL2002 dataset are provided here: https://github.com/teropa/nlp/tree/master/resources/corpora/conll2002. (We note that the official name of the dataset is CONLL-2002, although the publication reference is 2003.)

The files available in the CoNLL corpus include train and test data for the three parts of the CoNLL-2002 shared task:

1. esp.testa: Spanish test data for the development stage
2. esp.testb: Spanish test data
3. esp.train: Spanish train data
4. ned.testa: Dutch test data for the development stage
5. ned.testb: Dutch test data
6. ned.train: Dutch train data

We narrow the scope of this project to just the Spanish portion of the CoNLL-2002 dataset. (You are free to consider testing out the Dutch dataset on your own, as a contrastive case.)  Specifically, the **two files you will access are**:
1. For Training: 'esp.train'
2. For Testing: 'esp.testb'

We read the CoNLL corpus 2002 using *ConllCorpusReader* which is available in NLTK library. A *ConllCorpusReader* expects a data file with the following columnn types:

COLUMN_TYPES = ('words', 'pos', 'tree', 'chunk', 'ne', 'srl', 'ignore')
where
1. 'words': column type for words
2. 'pos': column type for Part of Speech
2. 'tree': column type for parse tree
3. 'chunk': column type for short phrases present in a given structure
4. 'ne': column type for named entities
5. 'srl': column type for Semantic Role Labeling
6. 'ignore': column type which can be ignored

All data files contain a single word per line with an associated named entity tag in the IOB2 format. The IOB2 format (short for inside, outside, beginning) is a common tagging format for tagging tokens in a chunking task in computational linguistics. The *I-* prefix before a tag indicates that the tag is inside a chunk. An *O* tag indicates that a token belongs to no chunk. The *B-* prefix before a tag indicates that the tag is the beginning of every chunk that immediately follows another chunk without *O* tags between them.

For example:
Alex (*B-PER*)
is (*O*)
going (*O*)
to (*O*)
Los (*B-LOC*)
Angeles (*I-LOC*)
in (*O*)
California (*B-LOC*).

Sentence breaks are encoded as empty lines.

In Part 3 we will see how the IOB2 format is used for Named Entity Recognition using the CoNLL2002 corpus.

**Run the code below to download the dataset.**

In [111]:
#@title Run this cell to download the dataset. (Do not edit the code.)
import nltk
from nltk.corpus.reader import ConllCorpusReader
nltk.download('conll2002')
import sklearn_crfsuite
print(nltk.corpus.conll2002.fileids())

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']


[nltk_data] Downloading package conll2002 to
[nltk_data]     /home/carson.johnson/nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!


# Part 2. Part of Speech Tagging
Part-of-speech (POS) tagging is commonly used technology in Natural Language Processing that categorizes words of a text (corpus) in terms of specific parts of speech, depending on the definition of the word and its context.

* A good read for POS tagging is **Section 17.2: Part-of-Speech Tagging of SLP3**.


## 2.0: Penn Treebank Corpus
We explore the Penn Treebank corpus downloaded in Part 1.

**Run the code below to read the Treebank tagged sentences.**

In [112]:
#@title Run this cell to read the Treebank tagged sentences. (Do not edit the code.)
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents())


#print the first five sentences along with tags
print("DISPLAYING SENTENCES")
print(nltk_data[:5])

#print each word with its respective tag for first five sentences
print("DISPLAYING WORDS")
for sent in nltk_data[:2]:
  for tuple in sent[:5]:
    print(tuple)

DISPLAYING SENTENCES
[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('of', 'IN'), ('this', 'DT'), ('British', 'JJ'), ('industrial', 'JJ'), ('conglomerate', 'NN'), ('.', '.'

## 2.1. Training and Test Samples
> The completion of this section prepares you to answer question 3, 4, 5 and 6 on **Project 1 Quiz questions**.

We now apply a 80:20 ratio to divide the training and test sets, and then compute the number of training and test samples for each set. Following this, we count the unique tags in the training data.

**Run the code below to obtain training and test samples.**

In [113]:
#@title Run this cell to obtain training and test samples.  (Do not edit the code.)
# split data into training and validation set in the ratio 80:20
train_set,test_set =train_test_split(nltk_data,train_size=0.80,test_size=0.20,random_state = 101)

# create list of train and test tagged words
train_tagged_words = [ tup for sent in train_set for tup in sent ]
test_tagged_words = [ tup for sent in test_set for tup in sent ]
print(len(train_tagged_words))
print(len(test_tagged_words))

# display the tagged words.
train_tagged_words[:5]

80310
20366


[('Drink', 'NN'),
 ('Carrier', 'NN'),
 ('Competes', 'VBZ'),
 ('With', 'IN'),
 ('Cartons', 'NNS')]

**Run the code below to count unique tags present in training data.**

In [114]:
#@title Run this cell to count unique tags present in training data.  (Do not edit the code.)
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

# check total words in vocabulary
vocab = {word for word,tag in train_tagged_words}

45
{'VBP', 'RP', '$', 'JJS', 'PRP$', '.', 'RBR', 'CD', 'RB', 'DT', "''", 'NNS', 'POS', 'EX', '``', 'MD', 'WP$', 'VBG', 'VBZ', 'PDT', 'FW', 'NN', 'WP', 'NNP', '-LRB-', 'WRB', 'UH', ',', 'PRP', '#', '-RRB-', 'JJR', 'JJ', 'CC', ':', 'LS', 'VBD', 'VB', 'VBN', 'TO', '-NONE-', 'NNPS', 'RBS', 'IN', 'WDT'}


We observe **45 tags in the training data**, including the -LRB- and -RRB- tags, which refer to "(" and ")", respectively. The resulting number of tags suggests that 3 out of 48 tags are not present in the training corpus. (We will return to a comparision of this output to that of the test data in Section 2.4.) A reference for the **Penn Treebank tagset for English language** is given here: https://universaldependencies.org/tagset-conversion/en-penn-uposf.html.

## 2.2. Hidden Markov Model

As mentioned before, Hidden Markov Model (HMM) is a probabilistic sequence model that leverages the principle of joint probability distribution. We calculate two probabilities for decoding algorithm *'Viterbi Algorithm'*:
1. Emission Probability
2. Transition Probability


> A good read is **Section 17.4.3 The components of an HMM tagger of SLP3**.

### 2.2.1. Emission Probability

Emission probability, also referred to as *operational likelihood*, expresses the probability that a word is generated from a tag. We first find the *tag list* and then we *count tags* to compute the emission probability.


### **Programming Question #1**:

**Complete the code below to define the function for computing Emission Probability. (Approximately 1 to 5 lines of code) (3 points)**

<mark>After you have completed the function and executed the tests, make sure to transfer your answer to the .py file provided along with this Notebook.</mark>

In [115]:
#@title Complete this code to define a function for computing Emission Probability.  (3 points)
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):

    """
    This function accepts word, tag and tagged words in training data to return count(w|tag) and count(tag)

    :param word: string
    :param tag: string
    :param train_bag: list of <_word_, _tag_>
    :return: count(w|tag) <integer>, count(tag) <integer>
    """
    # Write approximately 1 to 5 lines of code:

    # find the list (L) of pairs (_word_, _tag_) in train_bag having _tag_ equal to tag.
    L = [(_word_, _tag_) for (_word_, _tag_) in train_bag if _tag_ == tag]
    # find the count of elements in list (L) - total number of times the passed tag occurred in train_bag
    count = len(L)
    # find the number of times the word appears in pair <word, tag> of list (L)
    word_count = sum(1 for (_word_, _tag_) in L if _word_ == word)



    # return the word count given tag and the count of elements in the list of pairs
    return (word_count, count)

In [116]:
#@title Execute this cell to see if your changes work. (Do not change this code.)
# Test the function. DO NOT CHANGE CODE BELOW
try:
    assert word_given_tag('Drink', 'NN')
    assert word_given_tag('With', 'IN')
    count_w_given_tag, count_tag=word_given_tag('Drink', 'NN')
    print("Test Passed!")
    print("The output must look like:\n The output for count_w_given_tag is:  1 \n The output for count_tag is:  10510")
    print("OUTPUT:")
    print("The output for count_w_given_tag is: ", count_w_given_tag)
    print("The output for count_tag is: ", count_tag)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for count_w_given_tag is:  1 
 The output for count_tag is:  10510
OUTPUT:
The output for count_w_given_tag is:  1
The output for count_tag is:  10510


### 2.2.2. Transition Probability

We further calculate Transition Probability. HMM is based on the principle of random walk. An HMM-based solution obtains the probability of moving from one hidden state (tag-1 (t1)) to another hidden state (tag-2 (t2)) with a transition probability.


### **Programming Question #2**:

**Complete the code below to define the function for computing Transition Probability. (Approximately 1 to 5 lines of code) (3 points)**

<mark>After you have completed the function and executed the tests, make sure to transfer your answer to the .py file provided along with this Notebook.</mark>

In [117]:
#@title Complete the function below to define function for computing Transition Probability. (3 points)
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    """
    This function accepts two adjacent tags appearing in the text and tagged words in training data to return the count(t2|t1) and count(t1)

    :param t1: string
    :param t2: string
    :param train_bag: list of <_word_, _tag_>
    :return: count(t2|t1), count(t1)
    """

    # Write approximately 1 to 5 lines of code:

    # find the list (T) of tags present in the pairs of train_bag <_word_, _tag_>
    T = set([_tag_ for (_word_, _tag_) in train_bag])

    # count number of times t1 is present in the List (T)
    T1_count = sum(1 for (_word_, _tag_) in train_bag if _tag_ == t1)
    
    # count the number of times t2 appears after t1 in the List (T)
    T2_given_T1_count = sum(1 for i in range(len(train_bag)-1) 
                            if train_bag[i][1] == t1 and train_bag[i+1][1] == t2)



    # return count for t2 after t1, and the count for t1
    return (T2_given_T1_count, T1_count)

In [118]:
#@title Execute this cell to see if your changes work. (Do not change this code.)
# Test the function. DO NOT CHANGE CODE BELOW
try:
    assert t2_given_t1('VBZ', 'NN')
    assert t2_given_t1('NN', 'NN')
    assert t2_given_t1('IN', 'VBZ')
    print("Test Passed!")
    count_t2_t1, count_t1=t2_given_t1('VBZ', 'NN')
    print("The output must look like:\n The output for count_t2_t1 is: 468 \n The output for count_t1 is:  10510")
    print("OUTPUT:")
    print("The output for count_t2_t1 is: ", count_t2_t1)
    print("The output for count_t1 is: ", count_t1)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for count_t2_t1 is: 468 
 The output for count_t1 is:  10510
OUTPUT:
The output for count_t2_t1 is:  468
The output for count_t1 is:  10510


### 2.2.3. Transition Probability Matrix
A transition probability matrix (tags_matrix) is used to describe the probability of a tag, given the previous tag (to its left). Such transitions are accessed by sequence labeling processes, such as POS tagging and NER, to predict a tag for a word, given that of the previous word. For example, modal verbs (MD), e.g., *will*, are very likely to be followed by a verb in the base form  (VB), e.g., *race*. Thus, we expect the probability of the sequence MD VB to be higher than, for example, MD JJ. Below we create a *t x t* matrix *(M)* where *t* is the number of tags. Matrix *M(i,j)* represents the probability of tag *j* after tag *i*. We use this tags_matrix to store transition probabilities. Transition probabilities, coupled with their corresponding emission probabilities, are used in Section 2.3 below to compute state probabilities via this equation:

> *state probability = transition probability X emission probability*


**Run the code below to obtain Matrix M(i,j).**
> *Please note that it might take a couple of minutes to calculate the matrix.*




In [119]:
#@title Run this cell to obtain Matrix M(i,j). (Do not edit the code.)
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

print(tags_matrix)

[[0.0000000e+00 1.0194625e-02 9.2678407e-04 ... 0.0000000e+00
  8.9898057e-02 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 1.1904762e-02 ... 0.0000000e+00
  2.4404761e-01 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  3.3333335e-02 0.0000000e+00]
 [1.2655024e-04 1.2655024e-04 2.5689699e-02 ... 0.0000000e+00
  1.6957732e-02 3.9230576e-03]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  5.6179776e-03 0.0000000e+00]]


**Run the code below to convert the matrix into a dataframe for better readability.**
Note that the dataframe form of this table computed below contains the same data shown in the transition table above, but it is organized into a POS-tagged row/column format. The dataframe form is stored in the variable tags_df for use in Section 2.3.

In [120]:
#@title Run this cell to convert the matrix into a dataframe for better readability. (Do not edit the code.)
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
display(tags_df)

VBP        RP         $       JJS      PRP$         .       RBR  \
VBP     0.000000  0.010195  0.000927  0.000000  0.009268  0.006487  0.004634   
RP      0.000000  0.000000  0.011905  0.000000  0.059524  0.017857  0.000000   
$       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
JJS     0.007246  0.000000  0.014493  0.000000  0.000000  0.028986  0.000000   
PRP$    0.000000  0.000000  0.014610  0.012987  0.000000  0.000000  0.000000   
.       0.000000  0.000000  0.001290  0.001612  0.008062  0.000000  0.000645   
RBR     0.008850  0.000000  0.000000  0.000000  0.000000  0.044248  0.000000   
CD      0.003570  0.000000  0.000000  0.001071  0.000357  0.050696  0.000714   
RB      0.030052  0.000000  0.006969  0.000000  0.000871  0.042247  0.009146   
DT      0.001076  0.000000  0.007688  0.008610  0.000154  0.001230  0.001384   
''      0.003617  0.000000  0.000000  0.000000  0.001808  0.000000  0.000000   
NNS     0.092985  0.000411  0.000411  0.000000  0.000206  0.120757  0.001234   
POS     0.000000  0.000000  0.004525  0.019608  0.000000  0.010558  0.000000   
EX      0.216216  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
``      0.010526  0.000000  0.000000  0.000000  0.005263  0.000000  0.000000   
MD      0.000000  0.000000  0.000000  0.001353  0.000000  0.000000  0.001353   
WP$     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VBG     0.000000  0.021626  0.006055  0.000000  0.025087  0.015571  0.004325   
VBZ     0.000000  0.007026  0.006440  0.000585  0.009953  0.002927  0.003513   
PDT     0.000000  0.000000  0.000000  0.000000  0.130435  0.000000  0.000000   
FW      0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NN      0.003996  0.000381  0.000476  0.000190  0.000190  0.102664  0.000761   
WP      0.000000  0.000000  0.000000  0.000000  0.010753  0.000000  0.000000   
NNP     0.004056  0.000135  0.000135  0.000000  0.000000  0.051920  0.000000   
-LRB-   0.000000  0.000000  0.178218  0.000000  0.000000  0.000000  0.000000   
WRB     0.007194  0.000000  0.000000  0.000000  0.021583  0.000000  0.000000   
UH      0.000000  0.000000  0.000000  0.000000  0.000000  0.500000  0.000000   
,       0.008250  0.000000  0.007218  0.000773  0.002578  0.000000  0.001031   
PRP     0.180566  0.004351  0.000000  0.000000  0.000000  0.029007  0.000725   
#       0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
-RRB-   0.019231  0.000000  0.000000  0.000000  0.000000  0.115385  0.000000   
JJR     0.000000  0.000000  0.003236  0.000000  0.000000  0.064725  0.000000   
JJ      0.000638  0.000213  0.001701  0.000425  0.000000  0.022964  0.000213   
CC      0.013172  0.000000  0.020856  0.002744  0.013721  0.000000  0.002195   
:       0.013363  0.000000  0.006682  0.000000  0.002227  0.013363  0.002227   
LS      0.000000  0.000000  0.000000  0.000000  0.000000  0.555556  0.000000   
VBD     0.000000  0.014815  0.014403  0.000000  0.018930  0.007407  0.002881   
VB      0.000000  0.022994  0.008806  0.000489  0.038160  0.014677  0.007828   
VBN     0.000000  0.009977  0.005869  0.000000  0.015845  0.008803  0.000587   
TO      0.000000  0.000000  0.037123  0.000000  0.015661  0.000000  0.000580   
-NONE-  0.022355  0.000964  0.002313  0.000385  0.004240  0.089805  0.000964   
NNPS    0.045226  0.000000  0.000000  0.000000  0.000000  0.070352  0.000000   
RBS     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
IN      0.000127  0.000127  0.025690  0.004429  0.034169  0.002531  0.000886   
WDT     0.000000  0.000000  0.000000  0.002809  0.000000  0.000000  0.000000   

              CD        RB        DT  ...        LS       VBD        VB  \
VBP     0.008341  0.145505  0.112141  ...  0.000000  0.000927  0.000000   
RP      0.017857  0.035714  0.214286  ...  0.000000  0.000000  0.000000   
$       0.987296  0.000000  0.000000  ...  0.000000  0.000000  0.000000   
JJS     0.065217  0.043478  0.00

## 2.3. Viterbi Algorithm

The Viterbi Algorithm is a dynamic programming decoder for HMMs.
* A good read for Viterbi Algorithm is **Section 17.4.5: The Viterbi Algorithm in Chapter 17: Sequence Labeling for Parts of Speech and Named Entities in SLP3**.

In this section we will first define a function to compute a list of state probabilities (based on the matrix computed above), and then will define the Viterbi algorithm to use these state probabilities.


### **Programming Question #3**:

**Complete this function to produce a list of probabilities of different states for each tag. (Approximately 1 to 5 lines of code) (3 points)**

<mark>After you have completed the function and executed the tests, make sure to transfer your answer to the .py file provided along with this Notebook.</mark>

In [121]:
#@title Edit this code to produce a list of probabilities of different states for each tag. (3 points)
def compute_state_probability(key, word, T, state):
    """
    This function accepts key, word, list of tags T, the previous state (tag) and returns the list of probabilities of each tag in T
    being the next state


    :param key: int the position of the word in the sentence
    :param word: string
    :param T: List of unique tags
    :param tag: string
    :param p: List of probabilities for current iteration

    :return: List<state_probabilities>
    """
    p = []
    for tag in T:
        if key == 0:
            transition_p = tags_df.loc['.', tag]
        else:
            transition_p = tags_df.loc[state[-1], tag]

        # Write approximately 1 to 5 lines of code to compute emission and state probabilities:

        # calculate Emission probabilities
        word_count, tag_count = word_given_tag(word, tag)
        emission_p = word_count / tag_count if tag_count > 0 else 0

        # calculate state probabilities
        state_probability = transition_p * emission_p

        # add state probability in the list
        p.append(state_probability)

    return p

In [122]:
#@title Execute this cell to see if your changes work. (Do not change this code.)
# Test the function. DO NOT CHANGE CODE BELOW
try:
    p=[]
    state=[]
    T = list(set([pair[1] for pair in train_tagged_words]))
    assert compute_state_probability( 0, 'At', T, state)
    p=  compute_state_probability( 0, 'At', T, state)
    print("Test Passed!")
    print("The output must look like:\n The output for p is: [0, 0, 0, ..., 0.0003800179891361825,..., 0, 0, 0]")
    print("OUTPUT:")
    print("The output for p is: ", p)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like:
 The output for p is: [0, 0, 0, ..., 0.0003800179891361825,..., 0, 0, 0]
OUTPUT:
The output for p is:  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0003800179891361825, 0.0]


**Execute this code to define the function for Viterbi algorithm.**

In [123]:
#@title Run this cell to define the function for Viterbi Algorithm. (Do not edit the code.)
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation

        p=compute_state_probability(key, word, T, state)
        pmax = max(p)

        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

**Execute this code to test the Viterbi algorithm on a few sample sentences of test dataset**

In [124]:
#@title Run this cell to test the Viterbi algorithm. (Do not edit the code.)
#Once complete, test_subset_base contains the human-labeled
#tags corresponding toe the words in test_tagged_words.
random.seed(1234)      #define a random seed to get same sentences when run multiple times

# choose 10 random numbers
rndom = [random.randint(1,len(test_set)) for x in range(10)]

# list of 10 sents on which we test the model
test_subset = [test_set[i] for i in rndom]

# list of tagged words
test_subset_base = [tup for sent in test_subset for tup in sent]

# list of untagged words
test_subset_words = [tup[0] for sent in test_subset for tup in sent]

# list of tags in test subset
test_subset_tags = [tup[1] for sent in test_subset for tup in sent]

## 2.4. Viterbi Evaluation for POS Tagging
> The completion of this section prepares you to answer question 7 and 8 on **Project 1 Quiz questions**.

The Viterbi algorithm first sets up a lattice with one column for each word, and rows for tags where each cell represents each tag. Each cell $\mathrm{t_i}$ of the lattice represents the probability of that the HMM is in a given state after seeing the the previous observations (words: $\mathrm{w_1, w_2, ..., w_{i-1}}$) and passing through the most probable cell sequence $\mathrm{t_1, t_2, ..., t_{i-1}}$. The value of each cell is computed by recursively taking the most probable path (the maximum over all possible previous state sequences) that could lead us to this cell.

* A good read is **17.4.5 The Viterbi Algorithm in SLP3**.

One way to evaluate the accuracy of sequence labeling problems is to compare the output tags (tagged_seq) for an input sequence (test_subset_words) to a human-labeled sequence that we already know to be correct (test_subset_base). Here, we will verify only 10 sentences to check the accuracy as verification of the entire test set takes a very long time. Technically this notion of "accuracy" is referred to as "precision" (how many answers are correct out of the total number of answers), which we will learn more about in Project 2 and in the Evaluation module in the final segment of this course. (We will refer to it as "precision" further below.)

**Execute this code to test 10 sentences to find accuracy and time taken.**

> *Please note that it might take a couple of minutes to execute the Viterbi algorithm.*

> There could be some variability in Viterbi accuracy with multiple runs, but this does not adversely impact scoring for this project.

In [125]:
#@title Run this cell to test 10 sentences. (Do not edit this code.)
start = time.time()
tagged_seq = Viterbi(test_subset_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_subset_base) if i == j]


accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  41.03318548202515
Viterbi Algorithm Accuracy:  90.9090909090909


The Viterbi algorithm accuracy was computed above on 10 randomly chosen sentences. We exploit the resulting tagged sequence and test_subset_base to obtain more refined classification results, in terms of precision, recall, F1, and support. We will learn more about precision, recall, and F1 in Project 2. Below, the term *support* refers to the number of instances per tag. Note: The notions of micro/macro averaging will be discussed in more detail in an evaluation module later in the course.


In [126]:
#@title Run this cell to calculate the inventory of POS tags in the test corpus and the results of POS tagging on this corpus. (Do not edit this code.)
from sklearn.metrics import classification_report
actual_labels=[j for i, j in test_subset_base]
predict_labels=[j for i,j in tagged_seq]
#lab=['IN', 'JJ', 'NN', 'VB', 'RB', 'DT', 'CC' ] # We analyze some well-known POS tags.

results=classification_report(actual_labels, predict_labels)
print(results)

              precision    recall  f1-score   support

          ''       1.00      1.00      1.00         3
           ,       1.00      1.00      1.00        13
      -NONE-       1.00      0.93      0.97        15
           .       1.00      1.00      1.00        10
          CC       1.00      1.00      1.00         4
          CD       1.00      1.00      1.00         2
          DT       1.00      1.00      1.00        17
          IN       1.00      1.00      1.00        16
          JJ       0.86      0.86      0.86         7
         JJR       0.00      0.00      0.00         1
          MD       1.00      1.00      1.00         2
          NN       1.00      0.90      0.95        29
         NNP       1.00      0.87      0.93        23
        NNPS       1.00      1.00      1.00         1
         NNS       1.00      0.89      0.94         9
         PRP       1.00      1.00      1.00         4
          RB       1.00      0.91      0.95        11
         RBR       0.00    

/blue/cap4641/share/conda/envs/cap4641/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/blue/cap4641/share/conda/envs/cap4641/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/blue/cap4641/share/conda/envs/cap4641/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitali

## 2.5. For further study

You are free to acquire results and play with different combinations of POS tags for better inference. With these results, we leave open questions for you:
1. Observe the worst performing and the best performing POS tags. Analyze how the worst performing POS tags can be improved.
2. Explore the variation in the number of instances for each POS tag and how it affects the results.
3. Investigate whether a rule-based tagger, in-place of a probabilistic tagger, help in reducing the computational time (thereby, maintaining/ improving results).
4. Consider what it would take to explore the HMM-Viterbi algorithm for POS tagging for a different language.

# Part 3. Named Entity Recognition

Named-entity recognition (NER), also known as (named) entity identification, entity chunking, and entity extraction, is a subtask of information extraction that seeks to locate and classify named entities mentioned in unstructured text into predefined categories such as person names, organizations, locations.
* A good read is **Introduction to Chapter 17: Sequence Labeling for Parts of
Speech and Named Entities in SLP3**.

## 3.0 CoNLL2002 dataset

We will use the same CoNLL2002 corpus for the NER task that was downloaded in Part 1.2. We further analyze this dataset. There are as many as 6 fileids available in the dataset. In each file, every line represents a tagged word seperated by an empty space as *'word pos_tag NER_tag'*. Thus, three columns in every line consists of

1. The first column: Word
2. The second column: POS tag
3. The third column: Named entity

The data are represented with BIO tagging.
* A good read on BIO tagging is **Figure 17.3: Named Entities and Named Entity Tagging, showing IO, BIO, and BIOES taggings in Chapter 17: Sequence Labeling for Parts of Speech and Named Entities of SLP3**. For example, BIOES is an acronym that refers to:
* B: labels a token that begins a span
* I: labels a token inside a span
* O: labels a token outside of any span
* E: labels a token that ends a span and
* S: labels a single-token span

In this programming question, we focus on the simpler BIO tagging scheme: any token that begins a span of interest is labeled B, tokens that occur inside a span are labeled I, and any tokens outside of any span of interest are labeled O. We also augment the labels with named-entity tags:

1. 'B-LOC' : beginning location
2. 'B-MISC': beginning miscellaneous
3. 'B-ORG' : beginning organization
4. 'B-PER' : beginning person
5. 'I-LOC' : inside location
6. 'I-MISC': inside miscellaneous
7. 'I-ORG' : inside organization
8. 'I-PER' : inside person
9. 'O'.    : Outside of span of interest

**Run the code below to read the test file.**



In [127]:
#@title Run this cell to read the *test* file. (Do not edit this code.)
print(nltk.corpus.conll2002.raw('esp.testa')[:100])

Sao NC B-LOC
Paulo VMI I-LOC
( Fpa O
Brasil NC B-LOC
) Fpt O
, Fc O
23 Z O
may NC O
( Fpa O
EFECOM N


## 3.1 Training and Test Samples

We train and test a NER model using Spanish data from the CoNLL2002 dataset. The model is trained on *'esp.train'* and tested on *'esp.testb'*. We extract the dataset into two variables, *train_sents* and *test_sents*, respectively. The *iob_sents* function extracts the raw text of file in the form of a list of *tuples <word, pos_tag, ner_tag>* for each sentence.

We further explore the first annotated sentence in the training dataset to examine how POS tags and NE tags are assigned.  

**Execute the code below to obtain training and test data.**

In [128]:
#@title Run this cell to obtain training and test data for NER task. (Do not edit the code.)
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

train_sents[0]


[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

### **Programming Question #4**:

To understand the nature of the dataset, we now check the distribution of NER tags in the dataset for both training and test data.

**Complete this code to find the NER tag distribution in dataset. (Approximately 2 to 10 lines of code) (3 points)**

<mark>After you have completed the function and executed the tests, make sure to transfer your answer to the .py file provided along with this Notebook.</mark>

In [137]:
#@title Complete this code to find the NER tag distribution in dataset. (3 points)

def compute_tag_distribution(sents):
    """
    This function accepts a tuple <word, pos, ner> to return the NER_tag frequency distribution

    :param List of list of tuples <word, pos, ner>: list of list of (string, string, string)
    :return ner_tag_frequency: a dictionary of frequency of NER tags.
    """
    ner_tag_frequency=dict()

    # Write approximate 2-10 lines of code to create dictionay of <word, NER_tag> pairs.
    # In a list of sents given as a tuple <word, pos, ner>, count how many words
    # are there for each ner tag:
    
    for sent in sents:
        for (word, pos, ner) in sent:
            ner_tag_frequency[ner] = ner_tag_frequency.get(ner, 0) + 1

    return ner_tag_frequency

In [138]:
#@title Execute this cell to see if your changes work (Do not change this code.)
# Test the function. DO NOT CHANGE CODE BELOW
try:
    assert compute_tag_distribution(train_sents)
    assert compute_tag_distribution(test_sents)
    train_ner_tag_frequency=compute_tag_distribution(train_sents)
    test_ner_tag_frequency=compute_tag_distribution(test_sents)
    print("Test Passed!")
    print("The output for training corpora must look like: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, ...} ")
    print("OUTPUT for train_sents ner_tag_frequency is:", train_ner_tag_frequency)
    print("The output for test corpora must look like: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, ...} ")
    print("OUTPUT for test_sents ner_tag_frequency is:", test_ner_tag_frequency)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output for training corpora must look like: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, ...} 
OUTPUT for train_sents ner_tag_frequency is: {'B-LOC': 4913, 'O': 231920, 'B-ORG': 7390, 'B-PER': 4321, 'I-PER': 3903, 'B-MISC': 2173, 'I-ORG': 4992, 'I-LOC': 1891, 'I-MISC': 3212}
The output for test corpora must look like: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, ...} 
OUTPUT for test_sents ner_tag_frequency is: {'B-LOC': 1084, 'I-LOC': 325, 'O': 45355, 'B-ORG': 1400, 'B-MISC': 339, 'B-PER': 735, 'I-PER': 634, 'I-ORG': 1104, 'I-MISC': 557}


We observe that the NER tag distribution differs due to the size difference between the training and test corpora. However, the distributions follow a similar trend, with the highest number of tags being "O", etc. in both distributions.

## 3.2. Feature Extraction
> The completion of this section prepares you to answer question 9 and 10 on **Project 1 Quiz questions**.

As a starting point for the named-entity recognition task, we extract features for every word. A feature is an individual measurable property or characteristic of a word, for example, its part of speech (e.g., the POS for "eat" is a Verb). Choosing informative, discriminating and independent features is crucial for development of effective algorithms in pattern recognition, classification and regression. Producing accurate output for sequence labeling relies crucially on careful selection of features.

In addition to POS tags, other types of features may be extracted, e.g., word parts, simplified POS tags, lower/title/upper case flags, and features of nearby words.  To employ all such features in the task of Named Entity recognition, we start by converting them into sklearn-crfsuite format. Each sentence is converted into a list of dicts, i.e., a dictionary that maps feature names to feature indices. This is a very simple baseline; you certainly can do better.

We add following feature extraction steps for a given word and its neighbors (steps 1, 2, 3, and 4 refer to boolean values and 5 refers to a multi-category POS):
1. Word is lower-case (non-capitalized letters, e.g., "a")
2. Case of a word is `upper' (capitalized letters, e.g., "A")
3. Word is title (e.g., "Mr.")
4. Word is a digit (e.g., "10")
5. POS tag of word (e.g., "JJ")
6. Repeat steps from 2 to 5 for its neighbors

Tag frequency may influence the feature vector. Thus, features need to be weighted according to the number of times their corresponding tag appears. This introduces 'bias', an independent feature, which balances out the influence of varying tag frequency in the training corpus. Using 'bias' is a prevailing approach to address *feature bias* in adjusting the training loss for manually designed biased features. However, for this programming question, we make the simplifying assumption of uniform NER tag frequency distribution during model, e.g., ORG is assumed to occur as frequently as PERS. You are free to experiment independently with a weighted distribution for the 'bias' feature to train the model, but please retain a 'bias' setting of 1.0 for your final project submission.



### **Programming Question #5**

You will create a new function to define additional features for input words. Specifically, we add some basic NLP features like ‘postag’ and various features relating to letter case (lower, upper, etc.). Other than the given features, we can use any other meaningful handcrafted rule as additional feature, for example, position of a word.

**Complete the code below to define function for additional features. (Approximately 2 to 10 lines of code) (1 point)**

<mark>After you have completed the function and executed the tests, make sure to transfer your answer to the .py file provided along with this Notebook.</mark>

In [131]:
#@title Complete the code to define function for additional features. (1 point)
def word2features(sent, i):
    """
    This function accepts a sent (list of tuple in sentence) to return the additional features.

    :param sent: List of (string, string, string)
    :return features: dictionary
    """

    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:


        #find (i) previous word and (ii) postag for previous word.
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]

        # Add the following features for the previous word:
        # lowercased version of the word
        # value indicating whether word is title
        # value indicating whether word is uppercase
        # POS tag of the word
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:

        # Write approximately 8-10 lines of code to add features for the next word.
        #find (i) next word and (ii) postag for next word.
        word2 = sent[i+1][0]
        postag2 = sent[i+1][1]

        # Add the following features for the next word:
        # lowercased version of the word
        # value indicating whether word is title
        # value indicating whether word is uppercase
        # POS tag of the word
        features.update({
            '+1:word.lower()': word2.lower(),
            '+1:word.istitle()': word2.istitle(),
            '+1:word.isupper()': word2.isupper(),
            '+1:postag': postag2,
            '+1:postag[:2]': postag2[:2],
        })
    else:
        features['EOS'] = True

    return features

In [132]:
#@title Execute this cell to see if your changes work (Do not change this code.)
# Test the function. DO NOT CHANGE CODE BELOW
try:
    assert word2features(train_sents[3], 2)
    features=word2features(train_sents[3], 2)
    print("Test Passed!")
    print("The output must look like: {'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', ..., }")
    print("OUTPUT:")
    print(features)
except AssertionError as e:
    print("Test Failed")
    raise

Test Passed!
The output must look like: {'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', ..., }
OUTPUT:
{'bias': 1.0, 'word.lower()': 'del', 'word[-3:]': 'del', 'word[-2:]': 'el', 'word.isupper()': False, 'word.istitle()': False, 'word.isdigit()': False, 'postag': 'SP', 'postag[:2]': 'SP', '-1:word.lower()': 'petición', '-1:word.istitle()': False, '-1:word.isupper()': False, '-1:postag': 'NC', '-1:postag[:2]': 'NC', '+1:word.lower()': 'abogado', '+1:word.istitle()': True, '+1:word.isupper()': False, '+1:postag': 'NC', '+1:postag[:2]': 'NC'}


In [133]:
#@title Run this cell to define additional functions for feature extraction, to be used below. (Do not edit the code.)
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

### Extracting features

The input to feature extraction is a sentence. We obtain characteristics for each word in a sentence as a set of *features* in the form of a dictionary, as illustrated in Part 3.1. We now use sent2feature and sent2labels for feature extraction at the sentence level, recursively extracting features at the word level using the word2feature function.

**Execute the code below to extract features in the accepted format.**

In [134]:
#@title Run this cell to extract features in the accepted format. (Do not edit the code.)
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

X_train[0][3]

{'bias': 1.0,
 'word.lower()': ')',
 'word[-3:]': ')',
 'word[-2:]': ')',
 'word.isupper()': False,
 'word.istitle()': False,
 'word.isdigit()': False,
 'postag': 'Fpt',
 'postag[:2]': 'Fp',
 '-1:word.lower()': 'australia',
 '-1:word.istitle()': True,
 '-1:word.isupper()': False,
 '-1:postag': 'NP',
 '-1:postag[:2]': 'NP',
 '+1:word.lower()': ',',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'Fc',
 '+1:postag[:2]': 'Fc'}

## 3.3. Conditional Random Field (CRF) model

A Conditional Random Field (CRF) is a standard model for predicting the most likely sequence of labels that correspond to a sequence of inputs. CRF is a labeler in which the tag of the present word (denoted as yᵢ) depends only on the tag of just the previous word(denoted by yᵢ₋₁).
* A more detailed description of CRFs is provided at [Medium platform](https://medium.com/data-science-in-your-pocket/named-entity-recognition-ner-using-conditional-random-fields-in-nlp-3660df22e95c).
* Another good read on CRF is **Section 17.5: Conditional Random Fields of SLP3**.

Researchers often adopt CRFs over HMMs for Named Entity Recognition, given their incorporation of conditional probabilities, in contrast to the joint distributions used in HMMs. Named entity recognition is a computationally complex problem and modeling joint probabilities has disadvantages due to computational complexity. We note that it is also possible to apply CRFs to the problem of POS tagging, and we encourage you to try this on your own.

 * Additional information about CRF is available in **sklearn-crfsuite**: https://sklearn-crfsuite.readthedocs.io/en/latest/api.html#module-sklearn_crfsuite

**Execute the code below to train the model.**

> *Please note that it might take a couple of minutes for this calculation.*

In [135]:
#@title Run this code to train the model. (Do not edit the code.)
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=1.0,
    c2=1e-3,
    max_iterations=20,
    all_possible_transitions=True,
)
crf.fit(X_train, y_train);

## 3.4. CRF Evaluation for Named Entity Recognition
> The completion of this section prepares you to answer question 11 and 12 on **Project 1 Quiz questions**.

We further evaluate our model using sklearn metrics.

**Run the code below to test the model.**

In [136]:
#@title Run this cell to test the model. (Do not edit the code.)
from sklearn_crfsuite.metrics import flat_classification_report
import numpy as np

y_pred=crf.predict(X_test)

result=flat_classification_report(y_test, y_pred)
print(result)

              precision    recall  f1-score   support

       B-LOC       0.67      0.61      0.64      1084
      B-MISC       0.35      0.10      0.15       339
       B-ORG       0.66      0.76      0.71      1400
       B-PER       0.74      0.77      0.75       735
       I-LOC       0.49      0.24      0.32       325
      I-MISC       0.55      0.20      0.30       557
       I-ORG       0.63      0.79      0.70      1104
       I-PER       0.81      0.89      0.85       634
           O       0.99      1.00      0.99     45355

    accuracy                           0.95     51533
   macro avg       0.66      0.59      0.60     51533
weighted avg       0.95      0.95      0.95     51533



If computed correctly, your output above indicates an unusually high value for the 'O' (outside) prediction. More on this point in the question below.


## 3.5. For further study

With these observations, we give the outlook of this experiment and open problems in this area:
1. Consider the exploration of hybrid CRF approaches with other sequence-to-sequence models.
2. Examine the low scores for I-MISC and B-MISC, and possible ways to improve them.
3. Obtain results without 'O' NER tag.

# Part 4: Uploading responses for Grading

To upload your responses for grading, please follow the instructions provided in the `Project 1 - Sequence Labeling` page in Canvas.